# Initial Corpus Counts

*notes*

note regarding structure: this comes after I describe the bibliographical sources and the clean-up process for my origbib, but before I discuss the file sources.
Elements still needed: 
1. why discuss my sources in this depth
2. discussion of why even this bibliography is not "representative"
3. a good place to requote a source about the function of anthologies vs bibliographies
4. I need a naming structure for my charts/images. Mix of chapter/sections and Fig ids?
5. I need to figure out a way to render footnotes (base Markdown doesn't seem to work?)
6. I need to figure out integrating citations smoothly (seem to be resources on this)
7. Create some charts for by decade instead of year - more clear? Also curious!

*last update: April 18*

---------------------

Following the iterative process of cleaning my data, I had a bibliography of over 4970 short titles, along with their author and publication data. However, a variety of sources contributed to this data, and influenced the numbers. Before we move into searching for files, understanding the constitution of this foundational corpus is important because it influences what works we may find.

This notebook calculates and visualizes numbers in my original bibliography. This includes:
1. Publication numbers sorted by year
2. Publication numbers sorted by source bibliography
        a. Unique titles (ie, titles that appear only in one bibliography)
        b. Common titles (ie, titles that appear in more bibliographies)

There are a few things to note before diving into the numbers of this notebook. First, I delete titles that are dated 17--, a total of 5 titles). Secondly, the `pubdate_short` column is drawn from the first four characters of the `pubdate` column, meaning that works with a date range, such as 1727 "buck's antiquities or venerable remains" by Samuel Buck, published 1727-1740 according to Cox, is reduced to 1727. For many multi-volume titles---especially anthologies such as John Senex's *Modern Geography*, which can be published over several years or even decades---this decision creates a necessary distortion of the publication numbers for 214 titles.

I also take this opportunity to remind my readers (what am I, a game-show host?! ~change~ ) that not all of the titles are accurately recorded as first editions.

Let's jump in and look at some of the numbers, and try come up with some of my analysis as I do so.

In [1]:
# I had trouble updating the config (ie, my account) details for plotly and cufflinks
# Running command line/Anaconda prompt as admin solved the issue
import plotly.plotly as py
import cufflinks as cf

import pandas as pd
import numpy as np

In [2]:
from markdown.extensions import footnotes

In [3]:
origbib = pd.read_excel('finalbibs/origbib.v.16.xlsx', encoding='utf8')
len(origbib)

4970

In [4]:
# drop rows where the pubdate column is 17--
origbib = origbib[origbib.pubdate != '17--']
origbib['pubdate_short'] = pd.to_numeric(origbib['pubdate_short'])
len(origbib)

4965

In [5]:
origbib['pubdate_short'].value_counts().iplot(kind='bar', yTitle = 'Number of Titles',
                                          xTitle = 'Publication Year', title = 'Publication Counts from Original Bibliography',
                                          filename='dissertation/origbib_counts')

Let's start with a description of the chart. The highest peak is 144 titles in 1800; the late 1780s and through the 1790s are also higher on average. The peak at 1800 is followed by a significant drop in the early 1800s. Even without looking at which bibliography the various titles are from, the century divide is very clear: after 1800, the number of titles per year drops precipitously.

In [6]:
origbib['pubdate_short'].value_counts().sort_index()

1700    25
1701    18
1702    20
1703    21
1704    18
1705    23
1706    17
1707    28
1708    26
1709    20
1710    22
1711    24
1712    21
1713    10
1714    26
1715    26
1716    13
1717    27
1718    14
1719    17
1720    41
1721    12
1722    28
1723    22
1724    24
1725    26
1726    35
1727    29
1728    23
1729    23
        ..
1801    15
1802    12
1803    12
1804    14
1805    15
1806    11
1807    12
1808     6
1809    10
1810    16
1811     9
1812    10
1813    19
1814    50
1815    48
1816    41
1817    50
1818    53
1819    24
1820    19
1821    25
1822    23
1823    14
1824    26
1825    29
1826    13
1827    16
1828    20
1829    22
1830    25
Name: pubdate_short, Length: 131, dtype: int64

#  Bibliography by Source

I used 14 different sources for my bibliography (*see 1.XX for a more detailed description of each bibliography*). To better understand how each source contributes, we can compare several elements of these subcorpora:

1. Overall number of titles from each source
2. Publication dates from each source
3. Unique titles (titles only appear in one source)
4. Common titles (titles occur in multiple sources)

Because titles can appear in more than one source, totals calculated from the individual sources are inflated [~note passive] compared to the publication counts above.

As a reminder, our sources include:

In [7]:
listOfBibs = ['Robinson_W','Gove','BrynMawr', 'Leask', 'TEE', 'BDanth', 'Andrews',
 'Cox', 'BTW_W', 'Murray', 'Irish_McVeagh', 'BTW_EuroTour', 'NCCO_c19Trav', 'NCCO_TravelNarr']

## Overall Number of Titles from Each Source

In [8]:
# make a list of value_counts() results, then make them into a df

overallValueCountsList = []
for bib in listOfBibs:
    overallValueCountsList.append(origbib[bib].value_counts())

# number of titles from each bibliography
# transpose makes the col names (ie, the bibliographies) into index rows

overallValueCountsDF = pd.concat(overallValueCountsList, axis=1).transpose()
overallValueCountsDF.rename(columns={'x':'OverallTitleCounts'}, inplace=True)
overallValueCountsDF.sort_values(by=['OverallTitleCounts'])

,OverallTitleCounts
NCCO_TravelNarr,10
Robinson_W,41
BDanth,54
Leask,61
Andrews,65
TEE,67
Gove,79
Irish_McVeagh,93
NCCO_c19Trav,116
Murray,118


In [21]:
# compare the inflation from cross-referenced titles
overallValueCountsDF.sum() - len(origbib)

OverallTitleCounts    493
dtype: int64

Cox clearly dominates; of the 4965 titles in the `origbib` corpus, 4247 of them---85.5%---are in Cox. The next closest source, BrynMawr, is only at 198. The BTW_EuroTour is particularly interesting, as it contains 172 titles, but has a very limited scope: it focuses on travel between 1814-1818, excluding Britain and Ireland. If that number is at all indicative of usual publication numbers, then we would expect at least 1290 titles published from the end of 1800 to the end of 1830, rather than the 660 currently in my bibliography.

Murray's impact on the publication of travel writing is clear as well; the Murray bibliography, which includes only 'Non-European' travel texts, contributes more titltes than McVeagh's list of texts describing travels through Ireland, or Gove's list of imaginary travels.

The anthologis and critical works---such as BDanth, Leask, Andrews, and TEE---have less titles in general, as we might expect from works who do not aim at the cataloguing function of bibliographies. The unique cases of the NCCO collection are commented on earlier.

In [9]:
overallValueCountsDF.drop('Cox').sort_values(by=['OverallTitleCounts']).iplot(kind='bar', yTitle = 'Number of Titles',
                                          xTitle = 'Source', title = 'Publication Counts by Source (except Cox)',
                                          filename='dissertation/origbib_counts_by_source')

~Stefan note~ I also tried out a bar graph (below) that includes Cox and uses logs to ease visualization (`logy=True`), but I'm not sure if it is more helpful than the graph minus Cox above, and [the documentation on the plotly page](https://plot.ly/pandas/log-plot/) is a little...empty.

In [10]:
overallValueCountsDF.sort_values(by=['OverallTitleCounts']).iplot(kind='bar', yTitle = 'Number of Titles',
                                          xTitle = 'Source', title = 'Publication Counts by Source',
                                          filename='dissertation/origbib_counts_by_sourceWithCox', logy=True)

## Publication Counts by Year from Each Source

Just as our sources have different overall publication counts, they also contribute in different ways to the striation ~awk?~ of publication dates. 


In [11]:
tempBibList = []

for bib in listOfBibs:
    
    # select rows with only that bib source
    tempDF = origbib[origbib[bib] == 'x']
    
    # create value_counts() series, name for each bib source
    tempValueCounts = tempDF['pubdate_short'].value_counts().sort_index().rename(bib, inplace=True)
    
    # add the series to a list, to be used for concating
    tempBibList.append(tempValueCounts)

# concat all the series together on the index (ie, on the pubdate_short)
bibPubDF = pd.concat([x for x in tempBibList], axis =1, sort=False)

bibPubDF.sort_index(axis=1, inplace=True)

# preview the last 5 rows
bibPubDF.tail(5)

,Andrews,BDanth,BTW_EuroTour,BTW_W,BrynMawr,Cox,Gove,Irish_McVeagh,Leask,Murray,NCCO_TravelNarr,NCCO_c19Trav,Robinson_W,TEE
1826,NaN,NaN,NaN,2.0,1.0,1.0,NaN,3.0,NaN,5.0,NaN,NaN,2.0,4.0
1827,NaN,NaN,NaN,5.0,2.0,NaN,NaN,4.0,1.0,6.0,NaN,2.0,1.0,NaN
1828,NaN,NaN,NaN,6.0,5.0,NaN,NaN,2.0,2.0,7.0,1.0,3.0,1.0,4.0
1829,NaN,NaN,NaN,3.0,6.0,1.0,NaN,3.0,1.0,8.0,NaN,6.0,1.0,1.0
1830,NaN,NaN,NaN,6.0,3.0,1.0,NaN,5.0,NaN,5.0,1.0,6.0,3.0,2.0


In [12]:
bibPubDF.iplot(kind='bar', yTitle = 'Number of Titles', xTitle = 'Year of Publication',
               title = "Publication Counts from Each Source", filename='dissertation/origbib_counts_by_each_source')

In [13]:
bibPubDF.iplot(kind='bar', barmode='stack', yTitle = 'Number of Titles', xTitle = 'Year of Publication',
               title = "Publication Counts from Each Source", filename='dissertation/origbib_counts_by_each_source_stacked')

~Stefan note~ - There are some issues here with the colouring. The colours for Gove and McVeagh are way too similar, and they shouldn't be next to each other like that. In general, I think that the stacked chart is more clear than the other one above, but I'm wondering if I should collapse into decades?

The importance of Cox is clear, especially if you view the graph without his contributions (toggle off in above chart by clicking in key). He provides the only titles for 13 of the years before 1770. [footnote: These years are 1701, 1710, 1711, 1716, 1717, 1718, 1721, 1730, 1746, 1753, 1758, 1764, and 1769.] The years after 1810 are the most diverse from a source perspective, in part because of a few sources---such as BTW_EuroTour---that focus on that time period.



## Most Common Titles

My sources are a mixture of bibliographies, online databases, and critical works.

etc etc etc


In [14]:
# count which texts have the most sources
origbib['numberofbibentries'] = origbib[listOfBibs].eq('x').sum(axis=1)

In [22]:
origbib['numberofbibentries'].value_counts()

1    4609
2     259
3      65
4      25
5       6
6       1
Name: numberofbibentries, dtype: int64

In [24]:
origbib[origbib['numberofbibentries'] == 5][['title', 'author', 'pubdate_short']+listOfBibs]


,title,author,pubdate_short,Robinson_W,Gove,BrynMawr,Leask,TEE,BDanth,Andrews,Cox,BTW_W,Murray,Irish_McVeagh,BTW_EuroTour,NCCO_c19Trav,NCCO_TravelNarr
3234,Observations and reflections made in the cours...,"Piozzi, Hester Lynch",1789,x,NaN,x,NaN,NaN,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN
3918,Letters Written during a Short Residence in Sw...,"Wollstonecraft, Mary",1796,x,NaN,NaN,x,NaN,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN
4139,travels in the interior districts of africa,"park, mungo",1799,NaN,NaN,x,x,x,x,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN
4440,"travels in various countries of europe, asia, ...","clarke, edward daniel",1810,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,x,x
4476,Letters from a Lady to Her Sister during a Tou...,"Carter, Anne",1814,x,NaN,x,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,x,x,NaN
4700,Narrative of an expedition to explore the rive...,"Tuckey, James Kingston",1818,NaN,NaN,x,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,x,NaN


There is a steep drop from titles that are only in one bibliography to titles that are in 6. Which title is it?!

In [25]:
origbib[origbib['numberofbibentries'] == 6][['title', 'author', 'pubdate_short']+listOfBibs]


,title,author,pubdate_short,Robinson_W,Gove,BrynMawr,Leask,TEE,BDanth,Andrews,Cox,BTW_W,Murray,Irish_McVeagh,BTW_EuroTour,NCCO_c19Trav,NCCO_TravelNarr
3799,A Journey Made in the Summer of 1794,"Radcliffe, Ann",1795,x,NaN,x,NaN,NaN,x,x,x,x,NaN,NaN,NaN,NaN,NaN


Why might this book be so popular? Radcliffe is best-known as a literary author (works?)

## Titles that only appear in one bibliography

In [26]:
uniqueTitlesDF = origbib[origbib['numberofbibentries'] == 1]

In [27]:
uniqueTitlesDF

,pubdate,pubdate_mod,title,author,author_2,short_title,orig_merged,vol,galenum,author_3,...,OLD_NCCO,origbib_short_title,mainbib_short_title,match_column,htrc_id,ecco_id,gale_id,file_match,pubdate_short,numberofbibentries
1,1790,1790,Travels from the Cape of Good Hope,"Levaillant, Francrois","Helme, Elizabeth (translator)",Travels from the Cape of Good Hope,Travels from the Cape of Good Hope + Levaillan...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1790,1
2,1800,1800,Sketch of the Life and Literary Career of Augu...,"Kotzebue, August Friedrich Ferdinand von","Plumptre, Anne (translator)",Sketch of the Life and Literary Career,Sketch of the Life and Literary Career + Kotze...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1800,1
3,1802,1802,Travels in the Crimea. A History of the Embass...,"Struve, Johann Christian von","Godwin, Mary Jane (translator)",Travels in the Crimea,"Travels in the Crimea + Struve, Johann Christi...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1802,1
4,1812,1812,"Travels in Southern Africa, in the Years 1803,...","Lichtenstein, Martin Heinrich Carl","Plumptre, Anne (translator)","Travels in Southern Africa, in the Years 1803,...","Travels in Southern Africa, in the Years 1803,...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1812,1
5,1813,1813,Voyages and Travels in Various Parts of the Wo...,"Langsdorff, George Heinrich von","Plumptre, Anne (translator)",Voyages and Travels in Various Parts of the World,Voyages and Travels in Various Parts of the Wo...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1813,1
6,1813,1813,"Travels in the Morea, Albania, and Other Parts...","Pouqueville, François Charles Hugues Laurent","Plumptre, Anne (translator","Travels in the Morea, Albania, and Other Parts...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1813,1
7,1815,1815,Battle of Waterloo,"Eaton, Charlotte Anne",NaN,Battle of Waterloo,"Battle of Waterloo + Eaton, Charlotte Anne",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1815,1
8,1821,1821,authentic narrative of the shipwreck and suffe...,"bradley, eliza",NaN,authentic narrative of the shipwreck,authentic narrative of the shipwreck + bradley...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1821,1
9,1826,1826,memoirs of the margravine,"Craven, Lady Elizabeth",NaN,memoirs of the margravine,"memoirs of the margravine + Craven, Lady Eliza...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1826,1
10,1829,1829,memoirs of lady fanshawe,"fanshawe, lady ann",NaN,memoirs of lady fanshawe,"memoirs of lady fanshawe + fanshawe, lady ann",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1829,1


Using the same `value_counts` method as above, we'll recount to calculate how many unique values each bibliography contributed.

In [28]:
uniqueTitleCountsList = []

for bib in listOfBibs:
    uniqueTitleCountsList.append(uniqueTitlesDF[bib].value_counts())

uniqueTitleCountsDF = pd.concat(uniqueTitleCountsList, axis=1).transpose()
uniqueTitleCountsDF.rename(columns={'x':'UniqueTitleCounts'}, inplace=True)
uniqueTitleCountsDF.sort_values(by=['UniqueTitleCounts'])

,UniqueTitleCounts
Robinson_W,5
BDanth,5
NCCO_TravelNarr,5
TEE,13
Andrews,13
Leask,24
BrynMawr,43
NCCO_c19Trav,43
Gove,51
Murray,63


For ease, I want to combine this with my other `df` - `overallValueCountsDF`

In [29]:
bibOrigTitleCountsDF = uniqueTitleCountsDF.join(overallValueCountsDF)

In [30]:
bibOrigTitleCountsDF

,UniqueTitleCounts,OverallTitleCounts
Robinson_W,5,41
Gove,51,79
BrynMawr,43,198
Leask,24,61
TEE,13,67
BDanth,5,54
Andrews,13,65
Cox,4026,4247
BTW_W,77,137
Murray,63,118


Which bibliography had the most unique titles compared to overall titles?

In [31]:
bibOrigTitleCountsDF['UniqueToOverallRatio'] = bibOrigTitleCountsDF['UniqueTitleCounts']/bibOrigTitleCountsDF['OverallTitleCounts']

In [32]:
bibOrigTitleCountsDF.sort_values('UniqueToOverallRatio')

,UniqueTitleCounts,OverallTitleCounts,UniqueToOverallRatio
BDanth,5,54,0.092593
Robinson_W,5,41,0.121951
TEE,13,67,0.194030
Andrews,13,65,0.200000
BrynMawr,43,198,0.217172
NCCO_c19Trav,43,116,0.370690
Leask,24,61,0.393443
NCCO_TravelNarr,5,10,0.500000
Murray,63,118,0.533898
BTW_W,77,137,0.562044


Interesting things to note about the chart: 

- The anthologies have fairly low unique values. This might be expected, as anthologies anthologize (lol) works that critics consider to have influenced, defined, or exemplified a particular genre. (a qt on the purpose of anthologies would be useful here.)
- Cox still dominates. His work was the most detailed and thorough, working with a broad definition travel writing for his corpus.
- Some of these sources will naturally have more overlap than others. For example, some of the sources have a similar focus (such as women authors), and some sources relied on the others (BTW_W includes Leask, Cox, McVeagh, and Robinson_W, for example, or, more notably, BrynMawr serving as the foundation for NCCO_c19Trav).
- 